Example used in this notebook: 12as

In [ ]:
!wget https://files.rcsb.org/download/12AS.pdb
!wget https://files.rcsb.org/download/12AS.cif

Loading in CIF data

In [23]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import MMCIFParser
import pandas as pd

#https://biopython.org/docs/dev/api/Bio.PDB.mmcifio.html
#Initiate the parser, QUIET=True if you want to supress warning messages

#PDB parser
parser = PDBParser(QUIET=True)
#MMCIF parser
parser = MMCIFParser(QUIET=True)

#csv to map atom and residue types
df_weight = pd.read_csv('chemical_weights.csv')

file_name = '12AS.cif'
enzyme_name = '12AS'

#obtain the structure from the parser
structure = parser.get_structure(enzyme_name, file_name)

#Dictionary mapping residue names to distinct values
res_map = {'ALA':0, 'ARG':1, 'ASN':2, 'ASP':3, 'CYS':4, 'GLN':5, 'GLU':6, 'GLY':7, 'HIS':8, 'ILE':9, 'LEU':10, 'LYS':11, 'MET':12, 'PHE':13, 'PRO':14, 'SER':15, 'THR':16, 'TRP':17, 'TYR':18, 'VAL':19,'UNK':20}

#empty lists to store data
features_res = []
features_atom = []

#each structure can contain multiple models, e.g. NMR ensembles
for model in structure:
    #each model can contain multiple chains
    for chain in model:
        #each chain contains multiple residues
        for residue in chain:
            #Now we want to do some processing at residue level
            #E.g. remove UNK residues from our file
            #If residue is not in our pre-made weight file, we don't know the residue, so we skip it and log it.
            if residue.get_resname() not in df_weight.columns:
                
                # log the file name and residue name
                print(f"PDB file at -- <{file_name}> couldn't process -- <{residue.get_resname()}>")
                continue
            
            #Now lets do some processing at atom level
            for atom in residue:
                #We can access properties of residues and atoms
                #For more information see:
                #Residue module: https://biopython.org/docs/1.75/api/Bio.PDB.Residue.html
                #Atom module: https://biopython.org/docs/1.76/api/Bio.PDB.Atom.html
                atom_id = atom.get_id()
                
                #remove DNA atoms
                if atom_id not in df_weight['TYPE'].unique():
                    print(f"PDB file at -- <{file_name}> couldn't process -- <{atom_id}>")
                    continue
                    
                #grab coords
                x, y, z = atom.get_coord()
                
                #do specific processing based on atom name
                #e.g. selecting CA residues as your base residue for graph construction at residue level
                if atom_id == 'CA':
                    if residue.get_resname() in res_map:
                        features_res.append([x, y, z, res_map[residue.get_resname()]])
                    else:
                        features_res.append([x, y, z, 20.0])

                #you can remove hydrogen atoms
                #e.g. here we save any non H atoms to the atom list as hydrogen atom names always contain H
                if 'H' not in atom_id:
                    if residue.get_resname() in res_map:
                        features_atom.append([x, y, z, chemical_weight])
                    else:
                        features_atom.append([x, y, z, 30.0])



PDB file at -- <12AS.cif> couldn't process -- <AMP>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at -- <12AS.cif> couldn't process -- <HOH>
PDB file at 

This code gives us a array of only x, y, z coordinates and atom / residue types.

In [22]:
features_atom[0:10]

[[12.501, 39.048, 28.539, 3],
 [15.552, 39.41, 26.282, 3],
 [17.791, 39.281, 29.375, 3],
 [16.004, 36.186, 30.742, 3],
 [16.137, 34.313, 27.425, 3],
 [19.794, 35.327, 26.885, 3],
 [20.706, 33.656, 30.141, 3],
 [18.626, 30.599, 29.34, 3],
 [20.541, 30.119, 26.067, 3],
 [23.954, 30.474, 27.76, 3]]

Sometimes you might want to pre-process the file and remove any meta data for smaller storage. An easy trick:

In [ ]:
#open the original pdb file
with open('12AS.pdb', 'r') as infile:
    #open your output file to write to
    with open('12AS_processed.pdb', 'w') as outfile:
        #load all lines
        lines = infile.readlines()
        for line in lines:
            if line.startswith('ATOM'):
                outfile.write(line + '\n')
            else:
                continue
                